# Assignment 2
## by Vinit Saah

In [ ]:
import random
import threading
import time
import pynq.lib.rgbled as rgbled 
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

In [ ]:
def blink(t, d, n):
    '''
    Function to blink the LEDs
    Params:
      t: time to blink the LED for a given task
      d: duration (in seconds) for the LED to be on/off
      n: index of the LED (0 to 3)
    '''
    t = round(t/d) #converting the time into counter value
    for i in range(t):
        if n < 4:
            base.leds[n].toggle()
            time.sleep(d)
        elif n == 4:
            rgbled.RGBLED(n).write(0x2)
            time.sleep(d)
            rgbled.RGBLED(n).write(0x0)
            time.sleep(d)
    if n < 4:
        base.leds[n].off()

In [ ]:
blink(1, 0.1, 0)
blink(2,0.02,4)

In [ ]:

def bp_monitor(button_lock, btns):
    global stop_dining
    while stop_dining == False:
        time.sleep(0.01)
        if btns.read() != 0:
            button_lock.acquire(True)
            stop_dining = True
            print("Button {} is pressed stop_dining = \n", stop_dining)
            button_lock.release()
        


def dining_philosopher(f_right, f_left, button_lock, id_diner):   
    global stop_dining
    EATING = 1
    NAPPING = 2
    STARVING = 3
    
    count = 10
    
    diner_state = STARVING 
    
    eating_count = 0
    napping_count = 0
    starving_count = 0
    
    halt = False
    
    while halt == False:
        f_right_available = f_right.acquire(True)
        f_left_available = f_left.acquire(True)
        
        if f_right_available and f_left_available:
            diner_state = EATING
        else:
            if f_right_available:
                f_right.release()
            if f_left_available:
                f_left.release()
        
        if diner_state == EATING:
            eating_count += 1 
            print("Diner {} is EATING\n".format(id_diner))
            blink(1, 0.02,id_diner)
            f_right.release()
            f_left.release()
            diner_state = NAPPING
            time.sleep(0) #Yield  
    
        if diner_state == NAPPING:
            print("Diner {} is NAPPING\n".format(id_diner))
            napping_count +=1
            blink(0.5, 0.1,id_diner)
            diner_state = STARVING
            time.sleep(0) #Yield
    
        if diner_state == STARVING:
            starving_count += 1
            print("Diner {} is STARVING\n".format(id_diner))
            time.sleep(0)

        button_lock.acquire(True)
        if stop_dining == True:
            print("stop_dining is ", stop_dining)
            halt = True
        button_lock.release()
        
    print("Diner = ",id_diner, "eating count =", eating_count,"napping count =", napping_count, "starving_count =", starving_count)
        

stop_dining = False

if __name__ == "__main__":    
    philosophers_threads = list()
    fork_lock = list()    
    
    button_lock = threading.Lock()
    btns = base.btns_gpio
    b_t = threading.Thread(target = bp_monitor, args=(button_lock, btns))
    b_t.start()
    
    for index in range(5):
        f_l = threading.Lock()
        fork_lock.append(f_l)
        
    for i in range(5):    
        p_t = threading.Thread(target = dining_philosopher, args=(fork_lock[i], fork_lock[i-1],button_lock,i))
        philosophers_threads.append(p_t)
        p_t.start()
        
    for t in philosophers_threads:
        name = t.getName()
        t.join()
        print('{} joined'.format(name))

In [ ]:

def bp_monitor(button_lock, btns):
    global stop_dining
    while stop_dining == False:
        time.sleep(0.01)
        if btns.read() != 0:
            button_lock.acquire(True)
            stop_dining = True
            print("Button {} is pressed stop_dining = \n", stop_dining)
            button_lock.release()
        


def dining_philosopher(f_right, f_left, button_lock, id_diner):   
    global stop_dining
    EATING = 1
    NAPPING = 2
    STARVING = 3
    
    count = 10
    
    diner_state = STARVING 
    
    eating_count = 0
    napping_count = 0
    starving_count = 0
    
    halt = False
    
    while halt == False:
        f_right_available = f_right.acquire(False)
        f_left_available = f_left.acquire(False)
        
        if f_right_available and f_left_available:
            diner_state = EATING
        else:
            if f_right_available:
                f_right.release()
            if f_left_available:
                f_left.release()
        
        if diner_state == EATING:
            eating_count += 1 
            print("Diner {} is EATING\n".format(id_diner))
            rand_eat = random.randint(1,5)
            blink(rand_eat, 0.02,id_diner)
            f_right.release()
            f_left.release()
            diner_state = NAPPING
            time.sleep(0) #Yield  
    
        if diner_state == NAPPING:
            print("Diner {} is NAPPING\n".format(id_diner))
            napping_count +=1
            rand_nap = random.randint(1,5)
            blink(rand_nap, 0.1,id_diner)
            diner_state = STARVING
            time.sleep(0) #Yield
    
        if diner_state == STARVING:
            starving_count += 1
            print("Diner {} is STARVING\n".format(id_diner))
            time.sleep(0) #Yield

        button_lock.acquire(True)
        if stop_dining == True:
            print("stop_dining is ", stop_dining)
            halt = True
        button_lock.release()
        
    print("Diner = ",id_diner, "eating count =", eating_count,"napping count =", napping_count, "starving_count =", starving_count)
        

stop_dining = False

if __name__ == "__main__":    
    philosophers_threads = list()
    fork_lock = list()    
    
    button_lock = threading.Lock()
    btns = base.btns_gpio
    b_t = threading.Thread(target = bp_monitor, args=(button_lock, btns))
    b_t.start()
    
    for index in range(5):
        f_l = threading.Lock()
        fork_lock.append(f_l)
        
    for i in range(5):    
        p_t = threading.Thread(target = dining_philosopher, args=(fork_lock[i], fork_lock[i-1],button_lock,i))
        philosophers_threads.append(p_t)
        p_t.start()
        
    for t in philosophers_threads:
        name = t.getName()
        t.join()
        print('{} joined'.format(name))